### Задание
- Для любых видео восстановить траекторию движения (t вектор). Выполнить визуализацию. 
- Определить параметры которые влияют на "точность" определения вектора t
- Использовать решение на базе нейронных сетей. 
- Любые идеи. 

### Анализ
- При использовании каждого кадра ошибка вычислений будет очень быстро накапливаться => не будем обрабатывать каждый кадр. Будем оценивать движение с помощью calcOpticalFlowPyrLK и выбирать хорошие кадры с высокой резкостью и достаточным изменением сцены
- Будем использовать фильтр повышения резкости
- Изучим только sift, kaze и нейросетевой подход

In [1]:
import cv2
import numpy as np
import plotly.graph_objects as go

def extract_features(image, detector='sift', mask=None):
    if detector == 'sift':
        det = cv2.SIFT_create()
    elif detector == 'kaze':
        det = cv2.KAZE_create()
    
    kp, des = det.detectAndCompute(image, mask)
    
    return kp, des

def local_contrast_increase(frame):
    clahe = cv2.createCLAHE(clipLimit=5., tileGridSize=(8,8))
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l2 = clahe.apply(l)
    lab = cv2.merge((l2,a,b))
    
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def get_good_frames(video_path):
    good_frames = []
    last_image = None
    last_image_features = None
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Изменяем размер кадра
        height, width = frame.shape[:2]
        if width > height:
            new_width = 800
            new_height = int(height * (800 / width))
        else:
            new_height = 800
            new_width = int(width * (800 / height))
        frame_resized = cv2.resize(frame, (new_width, new_height))

        # Преобразуем в оттенки серого
        image_bw = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
        
        # Проверяем резкость кадра
        if cv2.Laplacian(image_bw, cv2.CV_64F).var() < 200:
            continue

        # Проверяем движение
        if last_image is not None:
            features, status, _ = cv2.calcOpticalFlowPyrLK(last_image, image_bw, last_image_features, None)
            if features is not None and status is not None:
                good_features = features[status == 1]
                good_features2 = last_image_features[status == 1]
                if len(good_features) > 0 and len(good_features2) > 0:
                    distance = np.average(np.abs(good_features2 - good_features))
                    if distance < 10:
                        continue

        good_frames.append(local_contrast_increase(frame))
        last_image = image_bw
        last_image_features = cv2.goodFeaturesToTrack(last_image, 500, 0.01, 10)

    cap.release()
    
    return good_frames

def match_and_filter_features(des1, des2, matching='BF', dist_threshold=0.75):
    if matching == 'BF':
        matcher = cv2.BFMatcher_create(cv2.NORM_L2)
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
    matches = matcher.knnMatch(des1, des2, k = 2)
    
    filtered_matches = []
    for m, n in matches:
        if m.distance <= dist_threshold * n.distance:
            filtered_matches.append(m)
    
    filtered_matches = sorted(filtered_matches, key=lambda x: x.distance)
    
    return filtered_matches

def position_experement(detector, matching, dist_threshold, K, video_path = './position/Castle.mp4'):
    is_first_frame = True
    poses = []
    good_frames = get_good_frames(video_path)
    
    for frame in good_frames:
        if is_first_frame:
            is_first_frame = False
            kp0, des0 = extract_features(frame, detector)
            continue
        
        kp1, des1 = extract_features(frame, detector)
        
        matches = match_and_filter_features(des0, des1, matching=matching, dist_threshold = dist_threshold )
        src_pts = np.float32([kp0[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp1[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        F, mask = cv2.findEssentialMat(src_pts, dst_pts, K, method=cv2.LMEDS, prob=0.999, threshold=1.0)     
        _, R, t, _ = cv2.recoverPose(F, src_pts, dst_pts, K, mask)
        poses.append((R, t))
        kp0, des0 = kp1, des1
    
    return poses

def create_trajectory(poses):
    trajectory = [np.array([0, 0, 0])]
    current_pose = np.eye(4)
    pose_cam = [np.array([[0, 0, 0],[0, 0, 0],[0, 0, 0]])]

    for R, t in poses:
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = t.T
        current_pose = np.dot(current_pose, T)
        trajectory.append(current_pose[:3, 3])
        pose_cam.append(current_pose[:3, :3])

    return np.array(trajectory),np.array(pose_cam)

def plot_trajectory(poses):
    trajectory, pose_cam = create_trajectory(poses)
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=trajectory[:, 0],
        y=trajectory[:, 1],
        z=trajectory[:, 2],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=2),
        name='Camera Trajectory'
    ))

    for i, (R, t) in enumerate(zip(pose_cam,trajectory)):
        # Направление камеры (ось Z камеры)
        camera_direction = R @ np.array([0, 0, 1])  # Направление оси Z камеры
        camera_direction_end = t + camera_direction * 0.5  # Конец вектора направления

        fig.add_trace(go.Scatter3d(
            x=[t[0], camera_direction_end[0]],
            y=[t[1], camera_direction_end[1]],
            z=[t[2], camera_direction_end[2]],
            mode='lines',
            line=dict(color='green', width=2),
            name=f'Camera Direction {i}' if i == 0 else None,
            showlegend=False if i > 0 else True
        ))

    fig.update_layout(
        title='Camera Motion Trajectory',
        scene=dict(
            xaxis_title='X-axis',
            yaxis_title='Y-axis',
            zaxis_title='Z-axis'
        ),
        showlegend=True
    )

    fig.show()

### Обработка видео классическими метдами
Обработаем алгоритмами sift и kaze и постараемся подобрать оптимальные параметры 
### Видео 1
Отрендерим видео с известной траекторией для более порстого сравнения и поиска ошибки.  
Матрица камеры известна. Точки распределены равномерно.

In [2]:
K = np.array([
    [4266, 0, 640],
    [0, 4266, 360],
    [0, 0, 1]
])

poses = position_experement('kaze', 'FLANN', 0.65, K, './position/Castle.mp4')

plot_trajectory(poses)


![video1_kaze.png](./img/video1_kaze.png)

### Видео 2
Видео облета здания Беларуськалий. Более сложное чем 1 т.к. большую часть времени четверть кадра занимает лес или река (хорошие точки распределены менее равномерно). Так же присутствует движение обьектов (автомобили). Камера матрицы подобрана вручную.
Источник: [https://www.youtube.com/watch?v=ff4DPbsEqw8](https://www.youtube.com/watch?v=ff4DPbsEqw8)

In [3]:
K = np.array([
    [1979, 0, 1920],
    [0, 1979 , 1080],
    [0, 0, 1]
])

poses = position_experement('sift', 'FLANN', 0.8, K, './position/Беларуськалий.webm')

plot_trajectory(poses)

![video2_sift.png](./img/video2_sift.png)

### Краткие наблюдения
- Лучший набор параметров соответствующий визуальному передвижению камеры: kaze+FLANN+0.65 и sift+FLANN+0.8. Подобраны для рендера, но применимы и для реального полета.
- Траектория стабильна на всем пути. Выбросов нет. Однако имеется небольшая накопленная погрешность.
- Для хороших результатов требуется точное определение матрицы камеры.

### Нейросетевое решение

In [2]:
import cv2
import torch
from kornia.feature import LoFTR
import numpy as np

def ml_position_experement_gpu(kamera_matrix, force_resize = False, video_path = './position/Castle.mp4'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    is_first_frame = True
    loftr = LoFTR(pretrained='outdoor').to(device)
    good_frames = get_good_frames(video_path)
    poses = []
    for frame in good_frames:  
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if force_resize:
            frame = cv2.resize(frame, (1920, 1080))
        frame = frame / 255.0
        if is_first_frame:
            prev_frame = frame
            is_first_frame = False
            continue
        
        frame_tensor = torch.from_numpy(frame).unsqueeze(0).unsqueeze(0).float().to(device)
        prev_frame_tensor = torch.from_numpy(prev_frame).unsqueeze(0).unsqueeze(0).float().to(device)
        input_dict = {"image0": frame_tensor, "image1": prev_frame_tensor}
        with torch.no_grad():
            correspondences = loftr(input_dict)

        mkpts0 = correspondences['keypoints0'].cpu().numpy().reshape(1, -1, 2)
        mkpts1 = correspondences['keypoints1'].cpu().numpy().reshape(1, -1, 2)

        F, mask = cv2.findEssentialMat(mkpts1, mkpts0, kamera_matrix, method=cv2.LMEDS, prob=0.999, threshold=1.0)     
        _, R, t, _ = cv2.recoverPose(F, mkpts1, mkpts0, kamera_matrix, mask)
        poses.append((R, t))
        prev_frame = frame
    
    return poses

### Видео 1 (Замок)

In [5]:
kamera_matrix = np.array([
    [4266, 0, 640],
    [0, 4266, 360],
    [0, 0, 1]
])

poses = ml_position_experement_gpu(kamera_matrix,  video_path = './position/Castle.mp4')

plot_trajectory(poses)

![video1_ml.png](./img/video1_ml.png)

### Видео 2 (Беларуськалий)

In [3]:
kamera_matrix = np.array([
    [990, 0, 960],
    [0, 990 , 540],
    [0, 0, 1]
])

# Не хватает памяти на GPU для обработки 4к видео, ресайзим до fullhd
poses = ml_position_experement_gpu(kamera_matrix, force_resize = True, video_path = './position/Беларуськалий.webm')

plot_trajectory(poses)

![video2_ml.png](./img/video2_ml.png)

### Выводы
- Повышение контрастности видео улучшает результаты обработки, особенно при использовании локального повышения контрастности.
- Выборка лучших кадров по резкости с достаточным движением (не менее 10 пикселей) помогает уменьшить накапливаемую ошибку и значительно улучшить результаты.
- Матрица камеры оказывает значительное влияние на конечный результат.
- LMEDS находит матрицу боле точно чем RANSAC.
- В большинстве ситуаций SIFT работает более точно, чем KAZE. Однако в некоторых случаях, например, на контрастных видео с городскими условиями, можно использовать KAZE.
- Удалось запустить LoFTR на CUDA. Модель требует больших вычислительных ресурсов по сравнению с классическими алгоритмами, но при использовании GPU может работать даже быстрее, чем классические алгоритмы на мощном CPU. LoFTR для природной местности работает значительно хуже чем классические алгоритмы. Для изображений со множеством контрастных элементов работает не хуже классических подходов.
- Все подходы показали точные результаты и могут быть использованы как инструменты одометрии. Выбросы и срывы отсутствуют, а траектория совпадает с наблюдаемой.